In [1]:
from qiskit import QuantumCircuit, transpile, QuantumRegister,ClassicalRegister ,execute, Aer
from qiskit.circuit.library import TwoLocal, EfficientSU2, RealAmplitudes
from qiskit.primitives import Estimator, Sampler
from qiskit_algorithms.minimum_eigensolvers import VQE 
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit.quantum_info import SparsePauliOp, Pauli, state_fidelity, random_statevector
from qiskit.extensions import RZGate, UnitaryGate
from qiskit.quantum_info import Operator, Statevector
from numpy import sqrt, diag, array,conjugate,transpose, append, dot, abs, pi, linalg, log, sqrt,arange, mean, std, exp
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from random import *

In [2]:
def Hamil_Schw(N,m,x):
    Hamil_Schw = 0
    Hamil_Schw_kin_list= []
    Hamil_Schw_mass_list= []
    Hamil_Schw_gauge_list= []
    for i in range(N-1):
        # Create mass terms
        Hamil_Schw_mass_list.append(("Z", [i], (-1)**(i+1)*m/2))
        if i == (N-2):
            Hamil_Schw_mass_list.append(("Z", [i+1], (-1)**(i+2)*m/2))
        # Create kinematic terms
        Hamil_Schw_kin_list.append((("XX", [i,i+1], x/2)))
        Hamil_Schw_kin_list.append((("YY", [i,i+1], x/2)))
        # Create gauge terms
        if i % 2 ==0:
            for j in range(i+1):
                Hamil_Schw_gauge_list.append(("Z", [j], -1/2))
        for j in range(i):
            for k in range(j+1,i+1):
                Hamil_Schw_gauge_list.append(("ZZ",[j,k],1/2))
    # Get in total
    Hamil_Schw = Hamil_Schw_kin_list+Hamil_Schw_mass_list+Hamil_Schw_gauge_list
    # Create Hamiltonian operator from the defined list
    Hamil_Schw = SparsePauliOp.from_sparse_list(Hamil_Schw, num_qubits=N)
    return Hamil_Schw


In [72]:
N = 4
m = 1
x = 1
del_t = 0.1
M = 3000

In [73]:
Hal =  Hamil_Schw(N,m,x)

# Classical solver
w,v = linalg.eig(Hal)
# print("Eigenvectors:",v)
# print("Eigenvalues:", w)

# Find ground state and energy in ensemble solutions
minimum=w[0]
min_spot=0
for i in range(1,2**N):
    if w[i]<minimum:
        min_spot=i
        minimum=w[i]                   
groundstate = v[:,min_spot]
# print("Ground State")
# print(groundstate)
# print("Ground Energy")
# print(minimum)

In [74]:
def H_evol(qc,n,M,m,x,del_t):
    N = qc.num_qubits
    for i in range(N-1):
        m_0 = 0.1
        # Create mass terms
        qc.rz(((-1)**(i+1)*m*n/M+m_0*(1-n/M))*del_t,i)
        if i == (N-2):
            qc.rz(((-1)**(i+2)*m*n/M+m_0*(1-n/M))*del_t,i+1)
        # Create gauge terms
        if i % 2 ==0:
            for j in range(i+1):
                qc.rz(-1*del_t,j)
        for j in range(i):
            for k in range(j+1,i+1):
                qc.rzz(1*del_t,j,k)
    for i in range(N-1):
        # Create kinematic terms
        qc.rxx(x*n/M*del_t,i,i+1)
        qc.ryy(x*n/M*del_t,i,i+1)
    return qc

In [75]:
# ground state is |1010...10>
qc = QuantumCircuit(N)
for i in range(qc.num_qubits):
    if i%2 ==1:
        qc.x(i)

for i in range(M):
    qc = H_evol(qc,i,M,m,x,del_t)

In [76]:
state_fidelity(Statevector(qc),Statevector(groundstate))

0.9957425061282443